<a href="https://colab.research.google.com/github/AkshayNagamalla/codeOptimisationGenAI/blob/main/flask_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U bitsandbytes
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install flask_cors
!pip install pyngrok
!pip install flask
!pip install request
!pip install jsonify
!pip install threading

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request
  Preparing metadata (setup.py) ... done
  Created wheel for jsonify: filename=jsonify-0.5-py3-none-any.whl size=1538 sha256=0c5c6c381de080173c7a8b257bf536da66c8e4bef41e203f1c43349824b20d77
  Stored in directory: /root/.cache/pip/wheels/4e/6d/b2/9cb590fb5654ff13457f49d4aa5b16469419681613c46a2ec1
Successfully built jsonify
ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from pyngrok import ngrok
import torch
from datetime import datetime

# Flask App Initialization
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Ngrok setup
ngrok.set_auth_token("your auth_key")#change auth token

# Load Model and Tokenizer
model_path = "G-210/code_optimization"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")


user_context = []

def process_query(query):
    """
    Process the query by appending it to the user context.
    Generates a new result based on the accumulated context.
    """
    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Append the new query and the current date to the user context
    user_context.append(f"{current_date}\nUser: {query}")
    full_context = "\n".join(user_context)

    # Prepare inputs for the model
    messages = [
        {"role": "user", "content": full_context},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate output
    output_tokens = model.generate(
        input_ids=inputs,
        max_new_tokens=128000,  # Adjust token length as needed
        use_cache=True,
        temperature=0.8,
        min_p=0.1,
    )

    # Decode the output tokens
    raw_output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # Filter out unwanted phrases
    unwanted_phrases = [
        "system"
        "Cutting Knowledge Date:",  # Add any phrases to exclude
        "Today Date:",
        "December 2023",
        "26 July 2024",
        "user"
    ]
    for phrase in unwanted_phrases:
        raw_output = raw_output.replace(phrase, "").strip()

    # Append only the user's query to the context to avoid repetition
    return raw_output


@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint to handle predictions based on user input.
    """
    data = request.get_json()
    user_input = data.get("i+nput", "")

    if not user_input:
        return jsonify({"error": "No input provided"}), 400

    try:
        # Generate a response using the model
        response = process_query(user_input)
        return jsonify({"prediction": response})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start the Ngrok tunnel and Flask app
if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"Ngrok tunnel is running at {public_url}")
    app.run(host="0.0.0.0", port=5000, threaded=True)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Ngrok tunnel is running at NgrokTunnel: "https://0b4e-34-143-163-234.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 20:24:58] "OPTIONS /predict HTTP/1.1" 200 -
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [27/Dec/2024 20:25:19] "POST /predict HTTP/1.1" 200 -
